In [2]:
import pandas as pd
import json
import os
from urllib.request import urlopen
import csv

# Download Data

In [ ]:
def json_to_df(json_data):
    df = pd.json_normalize(json_data["results"],["patient", "drug"] , meta=["serious", ["patient", "patientonsetage"], ["patient", "patientsex"], ["patient", "patientweight"], ], max_level = 5, errors='ignore')
    return df

In [67]:
data_dir = "../data"

In [114]:
drug_name = "021880"
# url = f"https://api.fda.gov/drug/event.json?search=receivedate:[20220101+TO+20221231]+AND+patient.drug.drugauthorizationnumb:{drug_name}+AND+occurcountry:US&limit=1000"
full_name = f"drug{drug_name}"

In [131]:
time = "20220101+TO+20221231"
url = f"https://api.fda.gov/drug/event.json?search=receivedate:[{time}]+AND+occurcountry:US&limit=1000"
full_name = f"drug_time_{time}"
print(url)

https://api.fda.gov/drug/event.json?search=receivedate:[20220101+TO+20221231]+AND+occurcountry:US&limit=1000


In [133]:
def fetch_url(url):
    dfs = []
    skip = 0
    while True:
        cur_url = "{}&skip={}".format(url, skip)
        try:
            response = urlopen(cur_url)
        except Exception as e:
            print(e)
            print(skip)
            break
        json_data = json.loads(response.read())
        if "error" in json_data:
            break
        for result in json_data["results"]:
            for drug in result["patient"]["drug"]:
                if "openfda" in drug.keys():
                    drug.pop("openfda")
        df = json_to_df(json_data)
        dfs.append(df)
        skip += 1000
    return pd.concat(dfs)

In [134]:
df = fetch_url(url)


In [129]:
df.shape

(68338, 29)

In [130]:
csv_name = f"{full_name}.csv"
csv_path = os.path.join(data_dir, csv_name)
df.to_csv(csv_path)

## Load Data

In [4]:
df = pd.read_csv("../data/2022q3_country.csv")

C:\Users\Nntraveler\AppData\Local\Temp\ipykernel_13980\3691262647.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/2022q3_country.csv")


In [5]:
df.head()

,Unnamed: 0,drugcharacterization,medicinalproduct,drugauthorizationnumb,drugstructuredosagenumb,drugstructuredosageunit,drugdosagetext,drugindication,actiondrug,activesubstance.activesubstancename,...,transmissiondate,seriousnessdeath,seriousnesslifethreatening,seriousnesshospitalization,seriousnessdisabling,seriousnesscongenitalanomali,seriousnessother,patient.patientonsetage,patient.patientsex,patient.patientweight
0,0,1,OMEPRAZOLE,212977.0,20.0,3.0,20 MILLIGRAM,Gastrooesophageal reflux disease,3.0,OMEPRAZOLE,...,20221026,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN
1,1,1,OMEPRAZOLE,212977.0,40.0,3.0,40 MILLIGRAM,Duodenal ulcer,3.0,OMEPRAZOLE,...,20221026,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN
2,2,1,RAMIPRIL,NaN,NaN,NaN,UNK,NaN,5.0,RAMIPRIL,...,20221026,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN
3,3,1,AMLODIPINE,NaN,NaN,NaN,UNK,NaN,5.0,AMLODIPINE BESYLATE,...,20221026,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN
4,4,1,TAMSULOSIN,NaN,NaN,NaN,UNK,NaN,5.0,TAMSULOSIN,...,20221026,2.0,2.0,2.0,2.0,2.0,1.0,NaN,NaN,NaN


## Sample

In [17]:
data_dir = "../data"
data_name = "drug018989_sample.json"
data_path = os.path.join(data_dir, data_name)
with open(data_path) as f:
    json_data = json.load(f)

In [18]:
for result in json_data["results"]:
    for drug in result["patient"]["drug"]:
        if "openfda" in drug.keys():
            drug.pop("openfda")


In [19]:
target_name = "drug018989_sample_no_openfda.json"
target_path = os.path.join(data_dir, target_name)
# 1. intialize json
# 2. for each step, update json_data
# 3. dump json_data
with open(target_path, "w") as f:
    json.dump(json_data, f)

In [6]:
def json_to_csv(json_data, target_path):
    data_file = open(target_path, 'w', newline='')
    csv_writer = csv.writer(data_file)
    
    count = 0
    for data in json_data:
        if count == 0:
            header = data.keys()
            csv_writer.writerow(header)
            count += 1
        csv_writer.writerow(data.values())
    
    data_file.close()

In [65]:
df.head()

,drugcharacterization,medicinalproduct,drugauthorizationnumb,drugdosagetext,drugdosageform,actiondrug,drugstartdateformat,drugstartdate,drugenddateformat,drugenddate,...,drugseparatedosagenumb,drugintervaldosageunitnumb,drugintervaldosagedefinition,drugindication,drugbatchnumb,drugadministrationroute,serious,patient.patientonsetage,patient.patientsex,patient.patientweight
0,1,LIPITOR,020702,UNK,FILM-COATED TABLET,5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,69,2,79.3
1,1,IBUPROFEN,018989,UNK,NaN,5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,69,2,79.3
2,1,ADVIL,018989,UNK,NaN,NaN,102,20140215,102,20140223,...,NaN,NaN,NaN,NaN,NaN,NaN,1,68,1,NaN
3,1,PREVISCAN,NaN,"10 MG, 1X/DAY",NaN,NaN,NaN,NaN,102,20140224,...,1,1,804,ARRHYTHMIA,NaN,NaN,1,68,1,NaN
4,2,METFORMIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,68,1,NaN


# Loading Dataframe

In [36]:
df = pd.read_csv(os.path.join(data_dir, "drug018989_sample_no_openfda.csv"))

In [37]:
df.head()

,safetyreportversion,safetyreportid,primarysourcecountry,occurcountry,transmissiondateformat,transmissiondate,reporttype,serious,receivedateformat,receivedate,...,patient/drug/0/drugindication,patient/drug/0/drugbatchnumb,patient/drug/0/drugstructuredosagenumb,patient/drug/0/drugstructuredosageunit,patient/drug/0/drugseparatedosagenumb,patient/drug/0/drugintervaldosageunitnumb,patient/drug/0/drugintervaldosagedefinition,patient/drug/0/drugadministrationroute,patient/drug/2/drugdosagetext,patient/drug/2/drugindication
0,1,10004874,US,US,102,20141002,1,2,102,20140312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10006370,FR,FR,102,20141002,1,1,102,20140313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10006649,US,US,102,20141002,1,2,102,20140313,...,PAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10006677,US,US,102,20141002,1,2,102,20140313,...,PAIN,H3154H,200.0,3.0,1.0,2.0,804.0,48.0,NaN,NaN
4,1,10007157,US,US,102,20141002,1,2,102,20140313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
